# Processing mixed beverage data
This python notebook uses a python library called [agate](http://agate.readthedocs.io/) to download, clean and process a monthly [Mixed Beverage Gross Receipts](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php) files from the Texas Comptroller's [data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/) to support [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

This first part of this pulls in python modules that we will use:

In [1]:
import agate
import re

/Users/christian/anaconda/envs/mixbev/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-03-29 09:59:47.596334. Please add timezone info to timestamps.


In [2]:
# this surpresses the timezone warning
import warnings
warnings.filterwarnings('ignore')

### Downloading the file

In [3]:
%%bash
## downloads the mixedbev file
## You have to set this URL based on the data center
curl -O https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_03_2017.CSV

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2493k  100 2493k    0     0  1537k      0  0:00:01  0:00:01 --:--:-- 1537k



There is supposedly a way to call a file from a [remote url](http://agate-remote.readthedocs.io/en/0.2.0/) into agate, but I use bash above to curl the file and store it locally instead.

Next, we'll use a bash command to peek at the data, which we know is a mess:

In [4]:
%%bash
head -n 5 MIXEDBEV_03_2017.CSV

"MB821424    ","ABI-HAUS                      ","959 N 2ND ST                  ","ABILENE             ","TX","79601","221","          ","2017/01", 000000523.40
"MB638028    ","ABILENE BEEHIVE INC           ","442 CEDAR ST STE A            ","ABILENE             ","TX","79601","221","          ","2017/02", 000002610.52
"MB543114    ","ABILENE BOWLING LANES INC     ","279 RUIDOSA AVE               ","ABILENE             ","TX","79605","221","          ","2017/02", 000000256.27
"MB933130    ","ABILENE CABARET LLC           ","1918 BUTTERNUT ST             ","ABILENE             ","TX","79602","221","          ","2017/02", 000000699.41
"N 037863    ","ABILENE COUNTRY CLUB          ","4039 S TREADAWAY BLVD         ","ABILENE             ","TX","79602","221","          ","2017/02", 000001801.63


### Study variables

This is where you set which file you are working with, and which month you want to study, etc.

First, we'll list the files in our directory that we have downloaded so far so we can get the filename

In [5]:
ls

MIXEDBEV_02_2017.CSV         counties.csv
MIXEDBEV_03_2017.CSV         headers.txt
Mixed beverages agate.ipynb  mixbev-env.txt
README.md


Then we set some values based on those.

- The `file` is the name of the file we want to process
- The `month_studied` is the YYYY/MM designation for the month before the file release. The file released in February has mostly records from January, but can also have any other month, so we set here the specific month we want. Note there is a check later on that counts the number of files by month, which is worth checking.
- The `city_studied` is a variable to study a specific city, if needed.
- The `county_studied` is a variable if you want a county other than Bastrop, Hays, Travis or Williamson.

In [6]:
# this is our source file, which may have been downloaded above
file = 'MIXEDBEV_03_2017.CSV'

# setting the month_studied var.
# This should be checked in the table below that counts records by month
month_studied = '2017/02'

# city studied needs to be ALL CAPS, as that's how the data comes
specific_city = 'AUSTIN'

# county studied should be ALL CAPS, as we set it to search that way
specific_county = 'CALDWELL'

### Data variables
These probalby won't change between analysis or data sets

In [7]:
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

### Import the file


In [8]:
# this imports the file specified above
mixbev_raw = agate.Table.from_csv(file, column_names, encoding='iso-8859-1', column_types=specified_types)

# prints table fields
print(mixbev_raw)

| column               | data_type |
| -------------------- | --------- |
| TABC Permit Number   | Text      |
| Trade Name           | Text      |
| Location Address     | Text      |
| Location City        | Text      |
| Location State       | Text      |
| Location Zip Code    | Text      |
| Location County Code | Text      |
| Blank                | Boolean   |
| Report Period        | Text      |
| Report Tax           | Number    |



### Clean up text fields

In [9]:
# This creates a new interim table with results of compute function
# that takes the four columns that need trimming and strips them
# adding them to the end of the table with new names
mixbev_trim = mixbev_raw.compute([
    ('Permit', agate.Formula(agate.Text(), lambda r: r['TABC Permit Number'].strip())),
    ('Name', agate.Formula(agate.Text(), lambda r: r['Trade Name'].strip())),
    ('Address', agate.Formula(agate.Text(), lambda r: r['Location Address'].strip())),
    ('City', agate.Formula(agate.Text(), lambda r: r['Location City'].strip()))
])

In [10]:
## shows the new columns added to the interim table
print(mixbev_trim)

| column               | data_type |
| -------------------- | --------- |
| TABC Permit Number   | Text      |
| Trade Name           | Text      |
| Location Address     | Text      |
| Location City        | Text      |
| Location State       | Text      |
| Location Zip Code    | Text      |
| Location County Code | Text      |
| Blank                | Boolean   |
| Report Period        | Text      |
| Report Tax           | Number    |
| Permit               | Text      |
| Name                 | Text      |
| Address              | Text      |
| City                 | Text      |



In [11]:
## creates new table with just stuff we need with clean names
# new_table = table.select(['3rd_column_name', '1st_column_name', '2nd_column_name'])
mixbev_cleaned = mixbev_trim.select([
    'Permit',
    'Name',
    'Address',
    'City',
    'Location State',
    'Location County Code',
    'Report Period',
    'Report Tax'
]).rename(column_names = {
    'Location State': 'State',
    'Location County Code': 'CountyCode',
    'Report Period': 'Period',
    'Report Tax': 'Tax'
})

In [12]:
## these are now the columns present in our new column
print(mixbev_cleaned)

| column     | data_type |
| ---------- | --------- |
| Permit     | Text      |
| Name       | Text      |
| Address    | Text      |
| City       | Text      |
| State      | Text      |
| CountyCode | Text      |
| Period     | Text      |
| Tax        | Number    |



In [13]:
# and this peeks at the data
# I did send this to_csv and made sure columns were trimmed
mixbev_cleaned.limit(5).print_table()

| Permit   | Name                 | Address              | City    | State | CountyCode | ... |
| -------- | -------------------- | -------------------- | ------- | ----- | ---------- | --- |
| MB638028 | ABILENE BEEHIVE INC  | 442 CEDAR ST STE A   | ABILENE | TX    | 221        | ... |
| MB543114 | ABILENE BOWLING L... | 279 RUIDOSA AVE      | ABILENE | TX    | 221        | ... |
| MB933130 | ABILENE CABARET LLC  | 1918 BUTTERNUT ST    | ABILENE | TX    | 221        | ... |
| N 037863 | ABILENE COUNTRY CLUB | 4039 S TREADAWAY ... | ABILENE | TX    | 221        | ... |
| MB200506 | ABILENE SEAFOOD T... | 1882 S CLACK ST      | ABILENE | TX    | 221        | ... |


### Create establishment column

We do this so we make sure we have single establishments instead of grouping trade names together from different addresses, like 'CHILI'S BAR & GRILL'.

In [14]:
# Concatenates the name and address
mixbev_cleaned_est = mixbev_cleaned.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda r: '%(Name)s %(Address)s' % r))
])

# Prints columns so you see it is there
print(mixbev_cleaned_est)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| City          | Text      |
| State         | Text      |
| CountyCode    | Text      |
| Period        | Text      |
| Tax           | Number    |
| Establishment | Text      |



In [15]:
# selects and prints Establishment to check what is looks like
mixbev_establishment = mixbev_cleaned_est.select('Establishment')
mixbev_establishment.print_table(max_column_width=80)

| Establishment                                      |
| -------------------------------------------------- |
| ABILENE BEEHIVE INC 442 CEDAR ST STE A             |
| ABILENE BOWLING LANES INC 279 RUIDOSA AVE          |
| ABILENE CABARET LLC 1918 BUTTERNUT ST              |
| ABILENE COUNTRY CLUB 4039 S TREADAWAY BLVD         |
| ABILENE SEAFOOD TAVERN 1882 S CLACK ST             |
| ABUELO'S BEVERAGE CORPORATION 4782 S 14TH ST       |
| ACE IN THE HOLE 133 EPLENS CT                      |
| AMNESIA, LLC. 1850 S CLACK ST                      |
| BILLIARDS PLUS 5495 S 7TH ST                       |
| BONZAI JAPANESE STEAK HOUSE 1802 S CLACK ST        |
| BREAKERS SPORTS BAR 1874 S CLACK ST                |
| BUFFALO WILD WINGS GRILL & BAR 1010 E OVERLAND TRL |
| BUFFALO WILD WINGS GRILL AND B 4401 RIDGEMONT DR   |
| CAHOOTS CATFISH & OYSTER BAR/J 301 S 11TH ST       |
| CHELSEA'S ST PUB 4310 BUFFALO GAP RD STE 1342      |
| CHILI'S GRILL & BAR 4302 S CLACK ST                |
| CHILIS G

### Import and merge counties lookup table
We do this to get county names

In [16]:
# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('counties.csv', column_types={'code': agate.Text()}).rename()

# peek at the column names
print(counties)

| column | data_type |
| ------ | --------- |
| id     | Number    |
| county | Text      |
| code   | Text      |



In [17]:
# peek at the data
counties.print_table()

| id | county    | code |
| -- | --------- | ---- |
|  1 | Anderson  | 001  |
|  2 | Andrews   | 002  |
|  3 | Angelina  | 003  |
|  4 | Aransas   | 004  |
|  5 | Archer    | 005  |
|  6 | Armstrong | 006  |
|  7 | Atascosa  | 007  |
|  8 | Austin    | 008  |
|  9 | Bailey    | 009  |
| 10 | Bandera   | 010  |
| 11 | Bastrop   | 011  |
| 12 | Baylor    | 012  |
| 13 | Bee       | 013  |
| 14 | Bell      | 014  |
| 15 | Bexar     | 015  |
| 16 | Blanco    | 016  |
| 17 | Borden    | 017  |
| 18 | Bosque    | 018  |
| 19 | Bowie     | 019  |
| 20 | Brazoria  | 020  |
| ... | ...       | ...  |


In [18]:
# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_cleaned_est.join(counties, 'CountyCode', 'code')

# check that the merge was succesful 
print(mixbev_joined)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| City          | Text      |
| State         | Text      |
| CountyCode    | Text      |
| Period        | Text      |
| Tax           | Number    |
| Establishment | Text      |
| id            | Number    |
| county        | Text      |



In [19]:
# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'Permit',
    'Name',
    'Address',
    'Establishment',
    'City',
    'State',
    'county',
    'Period',
    'Tax'
]).rename(column_names = {
    'county': 'County'
})

# peek at the column names
print(mixbev)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| Establishment | Text      |
| City          | Text      |
| State         | Text      |
| County        | Text      |
| Period        | Text      |
| Tax           | Number    |



In [20]:
# peek at the table
mixbev.limit(5).print_table()

| Permit   | Name                 | Address              | Establishment        | City    | State | ... |
| -------- | -------------------- | -------------------- | -------------------- | ------- | ----- | --- |
| MB638028 | ABILENE BEEHIVE INC  | 442 CEDAR ST STE A   | ABILENE BEEHIVE I... | ABILENE | TX    | ... |
| MB543114 | ABILENE BOWLING L... | 279 RUIDOSA AVE      | ABILENE BOWLING L... | ABILENE | TX    | ... |
| MB933130 | ABILENE CABARET LLC  | 1918 BUTTERNUT ST    | ABILENE CABARET L... | ABILENE | TX    | ... |
| N 037863 | ABILENE COUNTRY CLUB | 4039 S TREADAWAY ... | ABILENE COUNTRY C... | ABILENE | TX    | ... |
| MB200506 | ABILENE SEAFOOD T... | 1882 S CLACK ST      | ABILENE SEAFOOD T... | ABILENE | TX    | ... |


### Looking at dates of the records

Here we have to:
- create a tableset using group_by by the period
- create a table using aggregate function to count
- create a table to sort the period in reverse order
- Then print the sorted table (top 10 rows)

In [21]:
by_period = mixbev.group_by('Period')

period_totals = by_period.aggregate([
    ('count', agate.Count())
])

period_totals_sorted = period_totals.order_by('count', reverse=True)

# prints the table of period and number of records
period_totals_sorted.print_table(max_rows=None)


| Period  |  count |
| ------- | ------ |
| 2017/02 | 14,090 |
| 2017/01 |  1,423 |
| 2016/12 |    141 |
| 2016/11 |     52 |
| 2017/03 |     32 |
| 2016/10 |     26 |
| 2016/09 |     21 |
| 2016/08 |     13 |
| 2016/07 |      9 |
| 2016/05 |      8 |
| 2016/06 |      8 |
| 2016/04 |      5 |
| 2016/03 |      4 |
| 2015/02 |      4 |
| 2016/02 |      3 |
| 2014/12 |      3 |
| 2014/11 |      3 |
| 2015/12 |      2 |
| 2009/01 |      2 |
| 2014/10 |      2 |
| 2015/01 |      2 |
| 2015/03 |      2 |
| 2015/11 |      1 |
| 2011/05 |      1 |
| 2011/10 |      1 |
| 2016/01 |      1 |
| 2014/07 |      1 |
| 2015/10 |      1 |


The top value in the table above is typically the month before the reporting date. This shows how many records are filed for OTHER months as well. We want to make sure the top month value is included as the **month_studied** variable at the top of this file.

So, now we can filter the data to our specific month, which will use for the rest of the analysis:

In [22]:
## filters the records to our month_studied
mixbev_month = mixbev.where(lambda row: row['Period'] == month_studied)

## The number of records in our month
len(mixbev_month)

14090

## Top sales statewide

In [23]:
# groups the data based on Establishment and City
mixbev_grouped = mixbev.group_by('Establishment').group_by('County').group_by('City')
# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
state_summary_sorted = state_summary.order_by('total_sales', reverse=True)
# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                           | County  | City        | total_sales |
| ------------------------------------------------------- | ------- | ----------- | ----------- |
| ARAMARK SPORTS AND ENTERTAINME 211 AT AND T CENTER PKWY | Bexar   | SAN ANTONIO |  193,621.82 |
| GAYLORD TEXAN 1501 GAYLORD TRL                          | Tarrant | GRAPEVINE   |  125,633.17 |
| SALC, INC. 2201 N STEMMONS FWY FL 1                     | Dallas  | DALLAS      |   92,780.79 |
| LEVY RESTAURANTS 2500 VICTORY AVE                       | Dallas  | DALLAS      |   84,044.39 |
| AT&T STADIUM 1 LEGENDS WAY                              | Tarrant | ARLINGTON   |   82,089.87 |
| WLS BEVERAGE CO 110 E 2ND ST                            | Travis  | AUSTIN      |   79,804.43 |
| HILTON AMERICAS - HOUSTON 1600 LAMAR ST                 | Harris  | HOUSTON     |   78,343.97 |
| HOSPITALITY INTERNATIONAL, INC 23808 RESORT PKWY        | Bexar   | SAN ANTONIO |   76,712.72 |
| LEVY RESTAURANTS A

## Tops sales in a specific city

Uses the `city_studied` variable at the top of the workbook.

In [24]:
# Filters the data to the specified city
mixbev_city = mixbev_month.where(lambda row: row['City'] == specific_city)

# groups the data based on Establishment and City
city_grouped = mixbev_city.group_by('Establishment').group_by('City')
# computes the sales based on the grouping
summary = city_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
summary_sorted = summary.order_by('total_sales', reverse=True)
# prints the top 10 results
summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                      | City   | total_sales |
| -------------------------------------------------- | ------ | ----------- |
| WLS BEVERAGE CO 110 E 2ND ST                       | AUSTIN |   79,804.43 |
| ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE          | AUSTIN |   29,873.89 |
| 400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST        | AUSTIN |   28,395.13 |
| THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700     | AUSTIN |   27,372.64 |
| KUNG FU SALOON 11501 ROCK ROSE AVE STE 140         | AUSTIN |   23,468.15 |
| BARTON CREEK COUNTRY CLUB 8212 BARTON CLUB DR      | AUSTIN |   23,065.68 |
| TOP GOLF 2700 ESPERANZA XING                       | AUSTIN |   22,937.51 |
| SAN JACINTO BEVERAGE COMPANY L 98 SAN JACINTO BLVD | AUSTIN |   22,361.92 |
| ALAMO DRAFTHOUSE CINEMA 1120 S LAMAR BLVD          | AUSTIN |   22,324.66 |
| THE PALAZIO 501 E BEN WHITE BLVD                   | AUSTIN |   21,275.24 |


In [25]:
summary_sorted.limit(10).print_bars('Establishment', 'total_sales', width=80)

Establishment                                      total_sales
WLS BEVERAGE CO 110 E 2ND ST                         79,804.43 ▓░░░░░░░░░░░░░░░░
ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE            29,873.89 ▓░░░░░░          
400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST          28,395.13 ▓░░░░░░          
THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700       27,372.64 ▓░░░░░           
KUNG FU SALOON 11501 ROCK ROSE AVE STE 140           23,468.15 ▓░░░░░           
BARTON CREEK COUNTRY CLUB 8212 BARTON CLUB DR        23,065.68 ▓░░░░░           
TOP GOLF 2700 ESPERANZA XING                         22,937.51 ▓░░░░░           
SAN JACINTO BEVERAGE COMPANY L 98 SAN JACINTO BLVD   22,361.92 ▓░░░░            
ALAMO DRAFTHOUSE CINEMA 1120 S LAMAR BLVD            22,324.66 ▓░░░░            
THE PALAZIO 501 E BEN WHITE BLVD                     21,275.24 ▓░░░░            
                                                               +---------------+
                                              

## Top sales in a specific county

In [26]:
# Filters the data to the specified city
mixbev_county = mixbev_month.where(lambda row: row['County'].upper() == specific_county)

# groups the data based on Establishment and City
county_grouped = mixbev_county.group_by('Establishment').group_by('County')
# computes the sales based on the grouping
cn_summary = county_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
cn_summary_sorted = cn_summary.order_by('total_sales', reverse=True)
# prints the top 10 results
cn_summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                             | County   | total_sales |
| --------------------------------------------------------- | -------- | ----------- |
| GUADALAJARA MEXICAN RESTAURANT 1710 S COLORADO ST STE 110 | Caldwell |    1,149.78 |
| THE PEARL 110 N MAIN ST                                   | Caldwell |      617.74 |
| BRMJ, LLC 831 S COLORADO ST                               | Caldwell |      555.29 |
| MR TACO 1132 E PIERCE ST                                  | Caldwell |      511.81 |
| AUSTRIAN FARMS OF TEXAS LLC 4607 FM 86                    | Caldwell |        0.00 |


In [27]:
cn_summary_sorted.limit(10).print_bars('Establishment', 'total_sales', width=80)

Establishment                                             total_sales
GUADALAJARA MEXICAN RESTAURANT 1710 S COLORADO ST STE 110    1,149.78 ▓░░░░░    
THE PEARL 110 N MAIN ST                                        617.74 ▓░░░      
BRMJ, LLC 831 S COLORADO ST                                    555.29 ▓░░       
MR TACO 1132 E PIERCE ST                                       511.81 ▓░░       
AUSTRIAN FARMS OF TEXAS LLC 4607 FM 86                           0.00 ▓         
                                                                      +--------+
                                                                      0    2,000


## Bastrop

In [28]:
# Filters the data to the specified city
mixbev_bastrop = mixbev_month.where(lambda row: row['County'].upper() == 'BASTROP')

# groups the data based on Establishment and City
bastrop_grouped = mixbev_bastrop.group_by('Establishment').group_by('County')
# computes the sales based on the grouping
bastrop_summary = bastrop_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
bastrop_summary_sorted = bastrop_summary.order_by('total_sales', reverse=True)
# prints the top 10 results
bastrop_summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                     | County  | total_sales |
| ------------------------------------------------- | ------- | ----------- |
| LOST PINES BEVERAGE LLC 575 HYATT LOST PINES ROAD | Bastrop |   28,824.53 |
| OLD TOWN RESTURANT AND BAR/PIN 931 MAIN ST        | Bastrop |    4,089.61 |
| CHILI'S GRILL & BAR 734 HIGHWAY 71 W              | Bastrop |    2,659.63 |
| NEIGHBOR'S 601 CHESTNUT ST UNIT C                 | Bastrop |    1,849.13 |
| RED ROCK STEAKHOUSE & SALOON 101 S LENTZ ST       | Bastrop |    1,712.92 |
| LA HACIENDA RESTAURANT 1800 WALNUT ST             | Bastrop |    1,570.01 |
| VERANDA 910 MAIN ST                               | Bastrop |    1,420.40 |
| MORELIA MEXICAN CAFE 608 W ALAMO ST               | Bastrop |    1,225.89 |
| JALISCO MEXICAN RESTAURANT 244 HIGHWAY 290 W      | Bastrop |    1,135.31 |
| SP BASTROP THEATRE, LP 1600 CHESTNUT ST           | Bastrop |    1,113.87 |


## Hays

In [29]:
# Filters the data to the specified city
mixbev_hays = mixbev_month.where(lambda row: row['County'].upper() == 'HAYS')

# groups the data based on Establishment and City
hays_grouped = mixbev_hays.group_by('Establishment').group_by('County')
# computes the sales based on the grouping
hays_summary = hays_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
hays_summary_sorted = hays_summary.order_by('total_sales', reverse=True)
# prints the top 10 results
hays_summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                       | County | total_sales |
| --------------------------------------------------- | ------ | ----------- |
| THE MARC 120 E SAN ANTONIO ST                       | Hays   |    9,731.95 |
| BUCKS BACKYARD 1750 S FM 1626                       | Hays   |    8,278.98 |
| ZELICKS 336 W HOPKINS ST                            | Hays   |    8,003.01 |
| PLUCKERS WING BAR 105 N INTERSTATE 35               | Hays   |    6,545.02 |
| 54TH STREET RESTAURANT & DRAFT 1303 S INTERSTATE 35 | Hays   |    6,093.51 |
| CHIMY'S SAN MARCOS 217 E HOPKINS ST                 | Hays   |    6,088.22 |
| BLACK RABBIT SALOON / SHADE RO 127 E HOPKINS ST     | Hays   |    6,005.81 |
| TRATTORIA LISINA 13308 FM 150 W                     | Hays   |    5,696.07 |
| THE TAP ROOM & THE PORCH ON HO 129 E HOPKINS ST     | Hays   |    5,648.56 |
| CHUY'S SAN MARCOS 1121 N INTERSTATE 35              | Hays   |    5,637.78 |


## Travis

In [30]:
# Filters the data to the specified city
mixbev_travis = mixbev_month.where(lambda row: row['County'].upper() == 'TRAVIS')

# groups the data based on Establishment and City
travis_grouped = mixbev_travis.group_by('Establishment').group_by('County')
# computes the sales based on the grouping
travis_summary = travis_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
travis_summary_sorted = travis_summary.order_by('total_sales', reverse=True)
# prints the top 10 results
travis_summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                      | County | total_sales |
| -------------------------------------------------- | ------ | ----------- |
| WLS BEVERAGE CO 110 E 2ND ST                       | Travis |   79,804.43 |
| ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE          | Travis |   29,873.89 |
| 400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST        | Travis |   28,395.13 |
| THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700     | Travis |   27,372.64 |
| KUNG FU SALOON 11501 ROCK ROSE AVE STE 140         | Travis |   23,468.15 |
| BARTON CREEK COUNTRY CLUB 8212 BARTON CLUB DR      | Travis |   23,065.68 |
| TOP GOLF 2700 ESPERANZA XING                       | Travis |   22,937.51 |
| SAN JACINTO BEVERAGE COMPANY L 98 SAN JACINTO BLVD | Travis |   22,361.92 |
| ALAMO DRAFTHOUSE CINEMA 1120 S LAMAR BLVD          | Travis |   22,324.66 |
| THE PALAZIO 501 E BEN WHITE BLVD                   | Travis |   21,275.24 |


## Williamson

In [31]:
# Filters the data to the specified city
mixbev_williamson = mixbev_month.where(lambda row: row['County'].upper() == 'WILLIAMSON')

# groups the data based on Establishment and City
williamson_grouped = mixbev_williamson.group_by('Establishment').group_by('County')
# computes the sales based on the grouping
williamson_summary = williamson_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
williamson_summary_sorted = williamson_summary.order_by('total_sales', reverse=True)
# prints the top 10 results
williamson_summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                          | County     | total_sales |
| ------------------------------------------------------ | ---------- | ----------- |
| ALAMO DRAFTHOUSE CINEMA 14028 N HIGHWAY 183 BLDG F     | Williamson |   12,123.31 |
| JACK ALLEN'S KITCHEN 2500 HOPPE TRL                    | Williamson |    8,504.24 |
| TWIN PEAKS RESTAURANT 100 LOUIS HENNA BLVD             | Williamson |    8,482.93 |
| SALT TRADERS 2850 N INTERSTATE 35                      | Williamson |    7,894.40 |
| CHUY'S ROUND ROCK 2320 N INTERSTATE 35                 | Williamson |    7,647.44 |
| CHUY'S 4911 183A TOLL RD                               | Williamson |    6,960.83 |
| LUPE TORTILLA MEXICAN RESTAURA 4501 183A TOLL RD STE B | Williamson |    6,852.29 |
| THE BRASS TAP 204 E MAIN ST                            | Williamson |    6,455.18 |
| BJ'S RESTAURANT & BREWERY 1001 E WHITESTONE BLVD       | Williamson |    6,217.66 |
| GATEWAY BEVERAGES, INC 1101 WOODLAWN AVE            